In [0]:

#Yellow Trip Read
trip_input_df = spark.read.format("csv").option("header", True).load("/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-01.csv.gz")

display(trip_input_df)  # build it magic function in Databrciks
# trip_input_df.show() # python dataframe command

In [0]:
temp_df=trip_input_df.withColumnRenamed("store_and_fwd_flag","store_forward_flag")
display(temp_df)

In [0]:
from pyspark.sql.functions import col , substring,to_date, regexp_replace, expr , to_timestamp
# temp_df=trip_input_df \
# .withColumn("year_month" , regexp_replace(substring(col("tpep_pickup_datetime"),0,7) , "-" , "_"))

# or wrap in parenthesis
trip_df=(trip_input_df
.withColumn("year_month" , regexp_replace(substring(col("tpep_pickup_datetime"),0,7) , "-" , "_"))
.withColumn("pickup_datetime" , to_date(col("tpep_pickup_datetime"))) #  only extracts the date part (year, month, day) from a timestamp or string, and discards the time components (hour, minute, second, and millisecond) + chnage column's type from string to date
.withColumn("dropoff_datetime" , to_timestamp(col("tpep_dropoff_datetime"),"yyyy-MM-dd HH:mm:ss")) 
.withColumn("pickup_hour" , substring(trip_input_df["tpep_pickup_datetime"], 1, 2)) # another way to access column
.withColumn(
    "trip_pct",
    expr("try_divide(cast(tip_amount as float), cast(total_amount as float))")
  )
.limit(1000))


display(trip_df	)

In [0]:
#Zone_read
zone_df=spark.read.format("csv").option("header", True).load("/databricks-datasets/nyctaxi/taxizone/taxi_zone_lookup.csv")
display(zone_df)

In [0]:
full_df=trip_df.join(zone_df,trip_df.PULocationID==zone_df.LocationID,"left").drop("LocationID")
display(full_df)

In [0]:
# Did not work
full_df.write.mode("overwrite").format("delta").save("dbfs:/tmp/datasets/datakickstart/tables/nyctaxi")

In [0]:
full_df.write.mode("overwrite").saveAsTable("nyctaxi_sample")  # saves data in default schema

In [0]:
# new_df=spark.sql("select * from nyctaxi_sample")
# OR 
new_df=spark.table("nyctaxi_sample")
display(new_df)

In [0]:
# see lis of default mounts
dbutils.fs.help()


In [0]:
# See list of files in databricks-datasets
display(dbutils.fs.ls("dbfs:/databricks-datasets/"))


In [0]:
# See list of files in root
display(dbutils.fs.ls("dbfs:/"))

In [0]:
# Did not work
display(dbutils.fs.ls("dbfs:/Volumes/"))